In [1]:
import requests
import nltk 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd

### Get requests

**Aquí pedimos todos los mensajes de un usuario**

In [2]:
#data = requests.get('http://localhost:8080/John Wick').json()
url='http://localhost:8080/John Wick'

In [3]:
requests.get(url).json()

[{'text': 'Hey Mike, whats up??', 'userName': 'John Wick'},
 {'text': 'No, had to work. How was it?', 'userName': 'John Wick'},
 {'text': 'No way! 😮️', 'userName': 'John Wick'},
 {'text': 'I bet people went crazy', 'userName': 'John Wick'},
 {'text': 'Motherfucker! Hungover? 😵️', 'userName': 'John Wick'},
 {'text': 'Sure, boss.', 'userName': 'John Wick'},
 {'text': 'I do! I just dont feel appreciated in my job',
  'userName': 'John Wick'},
 {'text': 'I dunno...', 'userName': 'John Wick'},
 {'text': 'Maybe', 'userName': 'John Wick'},
 {'text': 'Thank you, boss! I feel much better...', 'userName': 'John Wick'}]

**Aquí pedimos todos los mensajes de un chat**

In [14]:
url='http://localhost:8080/chat/2/list'
chat2=requests.get(url).json()
chat2

[{'text': 'Sweety pie! How Ive longed to talk to you.',
  'userName': 'Mike Wazowski'},
 {'text': 'Not again, Michael.... 😒️', 'userName': 'Ellen Ripley'},
 {'text': 'Its Mike, honey bunny. Or doctor LOVE! And youre a sick little girl.',
  'userName': 'Mike Wazowski'},
 {'text': 'Im gonna report you to Human Resources again, Michael.',
  'userName': 'Ellen Ripley'},
 {'text': 'Cant handle me on your own?', 'userName': 'Mike Wazowski'},
 {'text': 'Id rather have a fucking alien crawling out of my ass',
  'userName': 'Ellen Ripley'},
 {'text': 'That would be hot... 🔥️', 'userName': 'Mike Wazowski'},
 {'text': 'Ok, youve got to stop drinking! Im calling HR.',
  'userName': 'Ellen Ripley'},
 {'text': 'I NEED VODKA LIKE A CAR NEEDS GAS', 'userName': 'Mike Wazowski'},
 {'text': 'Dude... Alcoholics Anonymous: https://www.aa.org',
  'userName': 'Ellen Ripley'}]

In [3]:
url='http://localhost:8080/chat/0/list'
chat0=requests.get(url).json()

In [11]:
type(chat0[0])

dict

In [35]:
sid = SentimentIntensityAnalyzer()
scores={}
for text in chat2:
    #print(chat2.index(text))
    scores['Message {} analysis'.format(chat2.index(text)+1)]=(text['userName']+': '+text['text']),(sid.polarity_scores(text['text']))

In [36]:
scores

{'Message 1 analysis': ('Mike Wazowski: Sweety pie! How Ive longed to talk to you.',
  {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}),
 'Message 10 analysis': ('Ellen Ripley: Dude... Alcoholics Anonymous: https://www.aa.org',
  {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}),
 'Message 2 analysis': ('Ellen Ripley: Not again, Michael.... 😒️',
  {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}),
 'Message 3 analysis': ('Mike Wazowski: Its Mike, honey bunny. Or doctor LOVE! And youre a sick little girl.',
  {'compound': 0.4451, 'neg': 0.178, 'neu': 0.54, 'pos': 0.282}),
 'Message 4 analysis': ('Ellen Ripley: Im gonna report you to Human Resources again, Michael.',
  {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}),
 'Message 5 analysis': ('Mike Wazowski: Cant handle me on your own?',
  {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}),
 'Message 6 analysis': ('Ellen Ripley: Id rather have a fucking alien crawling out of my ass',
  {'compound': -0.5423, 'neg'

In [4]:
df=pd.DataFrame(scores)
df=df[['neg', 'neu', 'pos']]
df=df.rename(columns={'neg':'Negative', 'neu':'Neutral', 'pos':'Positive'})
df

,Negative,Neutral,Positive
0,0.000,1.000,0.000
1,0.000,1.000,0.000
2,0.178,0.540,0.282
3,0.000,1.000,0.000
4,0.000,1.000,0.000
5,0.280,0.720,0.000
6,0.000,1.000,0.000
7,0.193,0.614,0.193
8,0.000,0.667,0.333
9,0.000,1.000,0.000


In [5]:
means=df.mean(axis = 0) 
means

Negative    0.0651
Neutral     0.8541
Positive    0.0808
dtype: float64

In [6]:
df=pd.DataFrame(means)

In [7]:
df

,0
Negative,0.0651
Neutral,0.8541
Positive,0.0808


In [8]:
df.columns = ['Scores']
df.index.name= 'Sentiment Metric'

In [9]:
df

,Scores
Sentiment Metric,
Negative,0.0651
Neutral,0.8541
Positive,0.0808


In [11]:
df.to_json

<bound method NDFrame.to_json of                   Scores
Sentiment Metric        
Negative          0.0651
Neutral           0.8541
Positive          0.0808>

In [13]:
sentiment_json=df.to_json

In [14]:
sentiment_json

<bound method NDFrame.to_json of                   Scores
Sentiment Metric        
Negative          0.0651
Neutral           0.8541
Positive          0.0808>

In [33]:
def getFinalMetric(chat):
    sid = SentimentIntensityAnalyzer()
    scores=[]
    conversation=[]
    for text in chat:
        conversation.append(text['userName']+': '+text['text'])
        scores.append(sid.polarity_scores(text['text']))
    df=pd.DataFrame(scores)
    df=df[['neg', 'neu', 'pos']]
    df=df.rename(columns={'neg':'Negative', 'neu':'Neutral', 'pos':'Positive'})
    means=df.mean(axis = 0) 
    df=pd.DataFrame(means)
    df.columns = ['Scores']
    df.index.name= 'Sentiment Metric'
    scores=df['Scores']
    final_sentiment_metric={}
    for s in scores:
        final_sentiment_metric['Negative']=scores[0]
        final_sentiment_metric['Positive']=scores[1]
        final_sentiment_metric['Neutral']=scores[2]
    return final_sentiment_metric

In [38]:
x=getFinalMetric(chat2)
x

{'Negative': 0.065100000000000005,
 'Neutral': 0.080800000000000011,
 'Positive': 0.85410000000000008}

In [42]:
scores['Complete chat sentiment Metric']=x
scores

{'Complete chat sentiment Metric': {'Negative': 0.065100000000000005,
  'Neutral': 0.080800000000000011,
  'Positive': 0.85410000000000008},
 'Final sentiment Metric': {'Negative': 0.065100000000000005,
  'Neutral': 0.080800000000000011,
  'Positive': 0.85410000000000008},
 'Message 1 analysis': ('Mike Wazowski: Sweety pie! How Ive longed to talk to you.',
  {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}),
 'Message 10 analysis': ('Ellen Ripley: Dude... Alcoholics Anonymous: https://www.aa.org',
  {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}),
 'Message 2 analysis': ('Ellen Ripley: Not again, Michael.... 😒️',
  {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}),
 'Message 3 analysis': ('Mike Wazowski: Its Mike, honey bunny. Or doctor LOVE! And youre a sick little girl.',
  {'compound': 0.4451, 'neg': 0.178, 'neu': 0.54, 'pos': 0.282}),
 'Message 4 analysis': ('Ellen Ripley: Im gonna report you to Human Resources again, Michael.',
  {'compound': 0.0, 'neg': 0.0, 'neu

**Solicitamos el informe de sentimiento de un chat.**

In [4]:
url='http://localhost:8080/chat/2/sentiment'
res=requests.get(url)
res

<Response [200]>

In [5]:
report_chat=res.json()
report_chat

{'Complete chat sentiment Metric': {'Negative': 0.0651,
  'Neutral': 0.08080000000000001,
  'Positive': 0.8541000000000001},
 'Message 1 analysis': ['Mike Wazowski: Sweety pie! How Ive longed to talk to you.',
  {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}],
 'Message 10 analysis': ['Ellen Ripley: Dude... Alcoholics Anonymous: https://www.aa.org',
  {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}],
 'Message 2 analysis': ['Ellen Ripley: Not again, Michael.... 😒️',
  {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}],
 'Message 3 analysis': ['Mike Wazowski: Its Mike, honey bunny. Or doctor LOVE! And youre a sick little girl.',
  {'compound': 0.4451, 'neg': 0.178, 'neu': 0.54, 'pos': 0.282}],
 'Message 4 analysis': ['Ellen Ripley: Im gonna report you to Human Resources again, Michael.',
  {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}],
 'Message 5 analysis': ['Mike Wazowski: Cant handle me on your own?',
  {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}],

### Post requests

**Añadimos un usuario con un mensaje y todos los datos del documento.**

In [4]:
url='http://localhost:8080/user/create'

In [5]:
new_us = {'userName': 'prueba sin mensaje'}

requests.post(url, data=new_us)

<Response [200]>

**Añadimos un usuario con un mensaje y todos los datos del documento.**

In [5]:
url='http://localhost:8080/adduser'

In [6]:
new_doc = {'userName': 'prueba user2', 
            'text': "dice cosas de nuevo"}

requests.post(url, data=new_doc)

<Response [200]>

**Añadimos un usuario con un mensaje a un chat existente.**

In [7]:
url='http://localhost:8080/chat/1/adduser'

In [9]:
new_doc = {'userName': 'Han Solo', 
            'text': "So, Leia, you think a princess and a guy like me..."}

requests.post(url, data=new_doc)

<Response [200]>